# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime

# Note that tstart and tend should span for at least one day because of the way the scripts are written.
tstart = '2018-07-27T00:00:00'
tend = '2018-07-28T00:00:00'

ttarget = '2018-07-27T20:21:00'
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')
ttarget = datetime.strptime(ttarget,'%Y-%m-%dT%H:%M:%S')



---
# Step 1, get the nominal RA/Dec Position for the Moon at the nominal target time

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(ttarget)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()

In [3]:
tend.timetuple().tm_yday

209

In [4]:
occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3:03d}:00:00:00 {4}:{5:03d}::00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday,    
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)

../orbit_engine/lunar_208to209.sh


---
# Step 2, get the PA position 

## We use this to figure out how to offset the FoV for the Moon

In [5]:

pastring = "./att2pa {0} {1} {2} -5 > lunar_{3}_{4}.pa".format(
    ttarget.strftime('%Y:%j:%H:%M:%S'),
    ra.to(u.deg).value, dec.to(u.deg).value,
    tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
outfile = '../orbit_engine/lunar_{0}to{1}_pa.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(pastring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)


../orbit_engine/lunar_208to209_pa.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced to produce the occultation files:

`./lunar_225to226.sh`

...and the one to get the PA position. **Remember that this is the MPS PA angle, so the SKY PA = 180 - MPA_PA. However, the code in nsutar_planning.io knows how to fix this and will return SKY PA values.**

`./lunar_225to226_pa.sh`

Check in the resulting output files in to github if you're working remotely.

`git add lunar_225to226.occ lunar_225to226.pa`

`git commit -m "Files for days 225-226"`

`git push`

Then pull this down to your local environment and continue below.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [6]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )
pafile= "../orbit_engine/lunar_{0}_{1}.pa".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

orbits = io.parse_occ(occfile)
sky_pa = io.parse_pa(pafile)


# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()
print('SKY PA angle: ', sky_pa)
orbits

SKY PA angle:  46.860378999999995


,visible,occulted
0,2018-07-27 00:46:58,2018-07-27 01:43:28
1,2018-07-27 02:23:37,2018-07-27 03:20:07
2,2018-07-27 04:00:16,2018-07-27 04:56:46
3,2018-07-27 05:36:55,2018-07-27 06:33:25
4,2018-07-27 07:13:34,2018-07-27 08:10:04
5,2018-07-27 08:50:13,2018-07-27 09:46:43
6,2018-07-27 10:26:52,2018-07-27 11:23:22
7,2018-07-27 12:03:31,2018-07-27 13:00:01
8,2018-07-27 13:40:10,2018-07-27 14:36:40
9,2018-07-27 15:16:49,2018-07-27 16:13:19


# Use SkyField to get the location of the Moon for each orbit:

This puts the output into the provided text file. This shows the *aim* time that was used to determine the pointing. You should slew while the source is occulted.

In [7]:
# Sanity check. Use this block to figure out which orbit(s) in the occultation file you want to use

import nustar_planning.lunar_planning as lunar_planning
sub = orbits.iloc[10:15].reset_index()
sub
#lunar_planning.position(sub, steps = 5, parallax_correction=True, load_path ='../../data')

,index,visible,occulted
0,10,2018-07-27 16:53:27,2018-07-27 17:49:58
1,11,2018-07-27 18:30:06,2018-07-27 19:26:37
2,12,2018-07-27 20:06:45,2018-07-27 21:03:16
3,13,2018-07-27 21:43:24,2018-07-27 22:39:55
4,14,2018-07-27 23:20:03,2018-07-28 00:16:34


In [10]:
# PA == sky_pa from above, Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)

# Note, this takes a few minutes to run!

outfile = 'lunar_{0}_{1}_lunar_eclipse.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

lunar_planning.position_shift(sub, parallax_correction=True, pa = sky_pa*u.deg, dt = 10*u.s, min_shift=6*u.arcmin,
                              outfile=outfile,pad_time=0*u.s, load_path ='../../data', Rmoon=882*u.arcsec)
print("Output is stored in: {}".format(outfile))

Output is stored in: lunar_208_209_lunar_eclipse.txt


In [9]:
import numpy as np
from astropy.coordinates import SkyCoord
outfile = 'lunar_{0}_{1}_lunar_eclipse.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )



f = open(outfile, 'r')
times = []
sky_pos = []
for line in f:
    if line.startswith('A'):
        continue



    fields = line.split()
    this_time = datetime.strptime(fields[0], '%Y:%j:%H:%M:%S')

    times = np.append(times, this_time)
    
    this_skypos = SkyCoord(ra=fields[1], dec=fields[2], unit ='deg')
    sky_pos = np.append(sky_pos, this_skypos)
    
f.close()

good_exp = 0.
all_exp = 0.
for ind in range(len(times)):
    if ind == 0:
        continue
    dt = (times[ind] - times[ind-1]).total_seconds()
    all_exp += dt
#    if (dt < 200):
#        continue
    dr = sky_pos[ind].separation(sky_pos[ind-1]).arcmin
    print(dt, dr)
    good_exp += dt

print()
print(good_exp, all_exp)


200.0 6.169740238141395
240.0 6.124131758248456
220.0 6.149805148448408
180.0 6.0508643817791175
150.0 6.372873460427363
140.0 6.463579886535234
130.0 6.343304147266612
120.0 6.048131680379883
110.0 6.188339516793559
110.0 6.2454329485403814
110.0 6.2155939119437
110.0 6.102250261125205
110.0 6.432556366689577
120.0 6.104269775740112
120.0 6.146168577216932
130.0 6.387669685455571
150.0 6.258761120262321
170.0 6.139818357051421
200.0 6.186741810179455
230.0 6.163332220877003
2749.0 139.87696148544956
200.0 6.175935322514455
240.0 6.033784224334077
220.0 6.047571953716519
180.0 6.393892350751576
160.0 6.350518366809937
140.0 6.447560964810457
130.0 6.33173534750058
120.0 6.041682216595398
110.0 6.185439725635106
110.0 6.24543515378124
110.0 6.219425231142679
110.0 6.1097700284720435
110.0 6.4433877779367705
120.0 6.118212405167445
120.0 6.165030415852125
130.0 6.009931023287715
140.0 6.34776248594072
170.0 6.224257428295845
200.0 6.2070264530770745
230.0 6.094232887470729
2749.0 139.834